In [1]:
#https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html

In [3]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(1)

In [5]:
SOS_TOKEN =0
EOS_TOKEN=1

class Lang:
    def __init__(self,name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word={0:'SOS',1:'EOS'}
        self.n_words =2
    
    def addSentence(self,sentence):
        for word in sentence.split(' '):
            self.addWord(word)
    
    def addWord(self,word):
        if word not in self.word2index:
            self.word2index[word]=self.n_words #Index of the said word
            self.index2word[self.n_words]=word
            self.n_words+=1
            self.word2count[word]=1
        else:
            self.word2count[word]+=1             

In [6]:
#Turn unicode to ASCII
def unicode2Ascii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD',s)
        if unicodedata.category(c) != 'Mn'
    )
#lowercase, trim and remove punctuation

def normalizeString(s):
    s = unicode2Ascii(s.lower().strip())
    s = re.sub(r"([.!?])",r"\1",s)
    s = re.sub(r"[^a-zA-Z.!?]+",r" ",s)
    return s 

In [9]:
#Read data file by splitting file into lines and then lines into pairs

def readLangs(lang1,lang2,reverse=False):
    path = '/home/cds/Desktop/rESUME PROJECTS/Machine Translation/data/eng-fra.txt'
    lines = open(path,encoding='utf-8').read().strip().split('\n')

    pairs = [[normalizeString(s) for s in l.split('\t')]for l in lines]

    if reverse:
        pairs=[list(reversed(p))for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)
    return input_lang, output_lang, pairs

In [12]:
#Filtering sentences by only taking sentences with max length less than 10
MAX_LENGTH =10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH and p[1].startswith(eng_prefixes)

def filterPairs(pairs):
    return[pair for pair in pairs if filterPair(pair)]

In [10]:
def prepareData(lang1,lang2,reverse=False):
    input_lang,output_lang,pairs = readLangs(lang1,lang2,reverse)
    print("Total sentence pairs ", len(pairs))
    pairs = filterPairs(pairs)
    print("Pairs retained", len(pairs))
    print("Counting words")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name,output_lang.n_words)
    return input_lang, output_lang, pairs

In [15]:
input_lang, output_lang,pairs = prepareData('eng','fra',True)
print(random.choice(pairs))

Total sentence pairs  135842
Pairs retained 11449
Counting words
fra 5713
eng 3802
['vous desobeissez aux ordres.', 'you re disobeying orders.']


In [39]:
#Seq2Seq model
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [18]:
#The decoder
class DecoderRNN(nn.Module):
    def __init__(self,hidden_size,output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size,hidden_size)
        self.gru = nn.GRU(hidden_size,hidden_size)
        self.out = nn.Linear(hidden_size,output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self,input,hidden):
        output = self.embedding(input).view(1,1,-1)
        output = F.relu(output)
        output,hidden = self.gru(output,hidden)
        output = self.softmax(self.out(output[0]))
        return output,hidden 
    
    def initHidden(self):
        return torch.zeros(1,1,self.hidden_size,device=device)

In [37]:
#Decoder with attention
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [20]:
#prepare training data
def indexesFromSentence(lang,sentence):
    return[lang.word2index[word]for word in sentence.split(' ')]

def tensorFromSentence(lang,sentence):
    indexes = indexesFromSentence(lang,sentence)
    indexes.append(EOS_TOKEN)
    return torch.tensor(indexes,dtype=torch.long, device=device).view(-1,1)

def tensorsFrompair(pair):
    input_tensor = tensorFromSentence(input_lang,pair[0])
    target_tensor = tensorFromSentence(output_lang,pair[1])
    return(input_tensor, target_tensor)


In [35]:
#Training

teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_TOKEN]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_TOKEN:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [22]:
import time 
import math 

def asMinutes(s):
    m = math.floor(s/60)
    s-=m*60
    return '%dm %ds' %(m,s)

def timeSince(since,percent):
    now = time.time()
    s = now-since
    es = s/(percent)
    rs = es-s
    return '%s (- %s)' %(asMinutes(s), asMinutes(rs))
    

In [25]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig,ax = plt.subplots()
    loc = ticker.MultipleLocator(base=0.2) #Puts ticks at regular intervals
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)
    plt.show()


def trainIters(encoder,decoder,n_iters,print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total=0 #Reset every print
    plot_loss_total=0 #Reset every plot

    encoder_optimizer = optim.SGD(encoder.parameters(),lr=learning_rate) #Try adam next
    decoder_optimizer = optim.SGD(decoder.parameters(),lr=learning_rate)
    training_pairs=[tensorsFrompair(random.choice(pairs)) for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1,n_iters+1):
        training_pair = training_pairs[iter-1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,decoder,encoder_optimizer,decoder_optimizer,criterion)
        print_loss_total+=loss
        plot_loss_total+=loss

        if iter%print_every ==0:
            print_loss_avg = print_loss_total/print_every
            print_loss_total=0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter%plot_every==0:
            plot_loss_avg = plot_loss_total/plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total=0

    showPlot(plot_losses)




In [42]:
#Evaluation

def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_TOKEN]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_TOKEN:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [47]:
def evaluate_randomly(encoder,decoder,n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('Input:',pair[0])
        print('Expected output:',pair[1])
        output_words,attentions = evaluate(encoder,decoder,pair[0])
        output_sentence = ' '.join(output_words)
        print('Prediciton:', output_sentence)
        print()

In [40]:
#Starting training
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words,hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size,output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1,attn_decoder1,75000,print_every=5000)

0m 49s (- 11m 36s) (5000 6%) 3.0881
1m 37s (- 10m 35s) (10000 13%) 2.5534
2m 26s (- 9m 47s) (15000 20%) 2.2946
3m 16s (- 8m 59s) (20000 26%) 2.0416
4m 5s (- 8m 10s) (25000 33%) 1.8048
4m 55s (- 7m 22s) (30000 40%) 1.6253
5m 45s (- 6m 34s) (35000 46%) 1.4702
6m 35s (- 5m 46s) (40000 53%) 1.3473
7m 25s (- 4m 57s) (45000 60%) 1.2228
8m 15s (- 4m 7s) (50000 66%) 1.1155
9m 5s (- 3m 18s) (55000 73%) 0.9739
9m 55s (- 2m 28s) (60000 80%) 0.9452
10m 45s (- 1m 39s) (65000 86%) 0.8351
11m 35s (- 0m 49s) (70000 93%) 0.8065
12m 25s (- 0m 0s) (75000 100%) 0.7327


/tmp/ipykernel_1235228/3457368554.py:12: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [48]:
evaluate_randomly(encoder1,attn_decoder1)

Input: c est lui n est ce pas ?
Expected output: he s the one isn t he?
Prediciton: he s the one one t you? <EOS>

Input: il a une assuetude a l heroine.
Expected output: he s addicted to heroin.
Prediciton: he s addicted to heroin. <EOS>

Input: nous sommes amis.
Expected output: we re friends.
Prediciton: we re friends. <EOS>

Input: nous sommes disponibles.
Expected output: we re available.
Prediciton: we re strong. <EOS>

Input: je ne suis vraiment pas tres bonne en francais.
Expected output: i m really not very good at french.
Prediciton: i m really not very good at french. <EOS>

Input: je suis pour toujours votre debiteur.
Expected output: i am forever in your debt.
Prediciton: i am still your your your debt. <EOS>

Input: elles sont en train de te chercher.
Expected output: they re looking for you.
Prediciton: they re looking for you. <EOS>

Input: elle essaie toujours de lui faire plaisir.
Expected output: she is always trying to please him.
Prediciton: she is always trying to